# Introduction

This notebook illustrates how to access a DB2 database on Cloud using Python by following the steps below:
1. Import the `ibm_db` Python library
2. Enter the database connection credentials
3. Create the database connection
4. Close the database connection
5. Create a table
6. Insert data into the table
7. Query data from the table
8. Retrieve the result set into a pandas dataframe
9. Close the database connection



#### Task 1: Import the `ibm_db` Python library

The `ibm_db` [API ](https://pypi.python.org/pypi/ibm_db/) provides a variety of useful Python functions for accessing and manipulating data in an IBM® data server database, including functions for connecting to a database, preparing and issuing SQL statements, fetching rows from result sets, calling stored procedures, committing and rolling back transactions, handling errors, and retrieving metadata.


We first import the ibm_db library into our Python Application
I didn't have installed the ibm library, so first, we installed it. 

In [9]:
#install the library. Execute just one time
#pip install ibm-db

In [2]:
import ibm_db

#### Task 2: Identify the database connection credentials

Connecting to dashDB or DB2 database requires the following information:
* Driver Name
* Database name 
* Host DNS name or IP address 
* Host port
* Connection protocol
* User ID (or username)
* User Password

In [3]:
#my credentials
#hide the cells
{
    "tags": [
        "hide_input",
        "hide_output"
    ]
}
dsn_hostname = "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net" 
dsn_uid = "zcl11296"       
dsn_pwd = "8qml-p1w2w8js19m"     

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            
dsn_port = "50000"               
dsn_protocol = "TCPIP"            

#### Task 3: Create the database connection

Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.

In [4]:
#hide output
{
    "tags": [
        "hide_output"
    ]
}
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=zcl11296;PWD=8qml-p1w2w8js19m;


In [6]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  zcl11296 on host:  dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net


Checking the Metadata

In [22]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB


In [23]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           DB2CLI.DLL
DRIVER_VER:            11.01.0405
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1257
CONN_CODEPAGE:         1208


We should close the connection, when we are done  
ibm_db.close(conn)

#### Task 4: Create a table in the database

In [7]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [8]:
#Construct the Create Table DDL statement
createQuery = 'create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, fname VARCHAR(20) NOT NULL, lname VARCHAR(20) NOT NULL , city VARCHAR(20) , ccode CHAR(2))' 

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)

In [9]:
print(createStmt)

#### Task 5: Insert data into the table

In [10]:
#Construct the query - replace ... with the insert statement
insertQuery = "insert into INSTRUCTOR(id, fname, lname, city, ccode) values (1, 'Ahuja','Rav','Toronto','CA'), (2, 'Chong','Raul','Toronto','CA'),(3, 'Vasudevan','Hima','Chicago','US')"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

#### Task 6: Query data in the table

In [32]:
#construct the select statement
selectQuery = 'select * from INSTRUCTOR'

#execute the select statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#fetch the data?? why it is just the first row? 
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Ahuja',
 1: 'Ahuja',
 'LNAME': 'Rav',
 2: 'Rav',
 'CITY': 'Toronto',
 3: 'Toronto',
 'CCODE': 'CA',
 4: 'CA'}

False


In [33]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Chong
 ID: 3  FNAME: Vasudevan


In [24]:
#seeing what  the fetch function does
insertQuery1 = "insert into INSTRUCTOR(id, fname, lname, city, ccode) values (4, 'test','test','Toronto','CA')"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery1)


In [26]:
#deleting the test row
deleteQuery = "delete from INSTRUCTOR where id = 4"

deleteStmt = ibm_db.exec_immediate(conn, deleteQuery)

In [30]:
selectQuery1 = 'select * from INSTRUCTOR'

#execute the select statement
selectStmt1 = ibm_db.exec_immediate(conn, selectQuery1)

#fetch the data?? why it is just the first row? 
ibm_db.fetch_both(selectStmt1)

{'ID': 1,
 0: 1,
 'FNAME': 'Ahuja',
 1: 'Ahuja',
 'LNAME': 'Rav',
 2: 'Rav',
 'CITY': 'Toronto',
 3: 'Toronto',
 'CCODE': 'CA',
 4: 'CA'}

In [28]:
ibm_db.fetch_both(selectStmt1)
#so fetch_both returns next row everytime the fetch_both is executed

{'ID': 2,
 0: 2,
 'FNAME': 'Chong',
 1: 'Chong',
 'LNAME': 'Raul',
 2: 'Raul',
 'CITY': 'Toronto',
 3: 'Toronto',
 'CCODE': 'CA',
 4: 'CA'}

Write and execute an update statement that changes the Rav's CITY to MOOSETOWN

In [38]:
updateQuery = "update INSTRUCTOR set city='Moosetown' where lname = 'Rav'" 

updateStmt = ibm_db.exec_immediate(conn, updateQuery)

In [41]:
selectQuery2 = 'select * from INSTRUCTOR'

#execute the select statement
selectStmt2 = ibm_db.exec_immediate(conn, selectQuery2)

ibm_db.fetch_both(selectStmt2)

{'ID': 1,
 0: 1,
 'FNAME': 'Ahuja',
 1: 'Ahuja',
 'LNAME': 'Rav',
 2: 'Rav',
 'CITY': 'Moosetown',
 3: 'Moosetown',
 'CCODE': 'CA',
 4: 'CA'}

#### Task 7: Retrieve data into Pandas 

In [43]:
import pandas
import ibm_db_dbi

In [44]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [45]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Rav'

In [46]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Ahuja,Rav,Moosetown,CA
1,2,Chong,Raul,Toronto,CA
2,3,Vasudevan,Hima,Chicago,US


In [47]:
pdf.shape

(3, 5)

#### Task 8: Close the Connection

In [48]:
ibm_db.close(conn)

True